# 自作モジュールを読み込むためのパスの設定
- おそらくノートブック形式でのみ必要

In [13]:
import sys
import os

sys.path.append(os.getcwd())
print(sys.path)

['/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/home/codehome/code', '/home/code', '/home/code']


In [14]:
import Preprocessing as pre
import pandas as pd
import datetime
from keras.models import load_model


## 推論に必要な情報の入力・パラメータの設定
- week_number_li : todayで設定した日付の次の日から21日分の手入力が必要
- temp_li : todayで設定した日付の当日から22日分の手入力が必要

In [15]:
today = '2024-05-04'
week_1 = 7
week_2 = 14
week_3 = 21
output_len = 50

week_number_li = [41,41,41,41,41,41,41,42,42,42,42,42,42,42,43,43,43,43,43,43,43]
temp_li = [18.6, 19.3, 19.6, 19.8, 20.7, 15.3, 16.4, 18.2, 19.2, 20.6, 21.0, 19.4, 20.0, 20.0, 20.0, 21.5, 22.5, 23.0, 23.5, 16.1, 18.4, 18.7]

data_path = '/home/code/data/daily_data_ver3.xlsx'
calender_path = '/home/code/data/calender_ver2.csv'
feature_path1 = '/home/code/4月4日_GRUモデル/1week/GRU_features.csv'
feature_path2 = '/home/code/4月4日_GRUモデル/2week/GRU_features.csv'
feature_path3 = '/home/code/4月4日_GRUモデル/3week/GRU_features.csv'
model_path1 = '/home/code/4月4日_GRUモデル/1week/gru_best_model_gru512_dense1024_loss0.0045.h5'
model_path2 = '/home/code/4月4日_GRUモデル/2week/gru_best_model_gru256_dense1024_loss0.0040.h5'
model_path3 = '/home/code/4月4日_GRUモデル/3week/gru_best_model_gru512_dense1024_loss0.0043.h5'
output_path = f'/home/code/4月4日_GRUモデル/pred_GRU_{today}.csv'

weekly_total_columns = ['Shipment', 'Teisyoku', 'Temp', 'High Temp', 'Low Temp', 'Status']
weekly_mean_columns = ['Week Shipment']
shift_unchange_name_columns = ['Status', 'Week Status', 'Week Shipment Mean', 'Week Status2']
shift_change_name_columns = ['Temp', 'Week Temp']
env_columns1 = ['Temp-'+str(week_1), 'Week Temp-'+str(week_1)]
env_columns2 = ['Temp-'+str(week_2), 'Week Temp-'+str(week_2)]
env_columns3 = ['Temp-'+str(week_3), 'Week Temp-'+str(week_3)]
cat_columns = ['Week Number']

drop_columns = ['Target', 'Week', 'Week Start', 'Week End', 'Week Status2', 'Status', 'Teisyoku', 'Week Teisyoku']

## データの前処理

In [16]:
df = pd.read_excel(data_path)
print(df.tail(5))
df_calender = pd.read_csv(calender_path, index_col=0, parse_dates=True)

df_week1 = df.copy()
df_week2 = df.copy()
df_week3 = df.copy()

df_week1 = (
        df_week1.pipe(pre.change_column_name)
        .pipe(pre.set_index_date)
        .pipe(pre.add_pred_rows, today=today, week_number_li=week_number_li, temp_li=temp_li, target_days=week_1)
        .pipe(pre.add_status, today=today, target_days=week_1, calender_path=calender_path)
        .pipe(pre.add_weekday)
        .pipe(pre.add_weekly_total, columns=weekly_total_columns)
        .pipe(pre.add_weekly_mean, columns=weekly_mean_columns) 
        .pipe(pre.add_week_status2)
        .pipe(pre.shift_unchange_name, shift_columns=shift_unchange_name_columns, shift_days=week_1)
        .pipe(pre.shift_change_name, shift_columns=shift_change_name_columns, shift_days=week_1)
        .pipe(pre.change_env_data, env_columns=env_columns1)
        .pipe(pre.add_target, target_days=week_1) # Week Shipmentをズラす前に作成することに注意
        .pipe(pre.add_target2, target_days=week_1) # Week Shipmentをズラす前に作成することに注意
        .pipe(pre.shift_unchange_name, shift_columns=['Week Shipment'], shift_days=-7) # Week Shipmentは1週間前のデータを使用
        .pipe(pre.categorize_columns, cat_columns=cat_columns)
)

df_week2 = (
        df_week2.pipe(pre.change_column_name)
        .pipe(pre.set_index_date)
        .pipe(pre.add_pred_rows, today=today, week_number_li=week_number_li, temp_li=temp_li, target_days=week_2)
        .pipe(pre.add_status, today=today, target_days=week_2, calender_path=calender_path)
        .pipe(pre.add_weekday)
        .pipe(pre.add_weekly_total, columns=weekly_total_columns)
        .pipe(pre.add_weekly_mean, columns=weekly_mean_columns) 
        .pipe(pre.add_week_status2)
        .pipe(pre.shift_unchange_name, shift_columns=shift_unchange_name_columns, shift_days=week_2)
        .pipe(pre.shift_change_name, shift_columns=shift_change_name_columns, shift_days=week_2)
        .pipe(pre.change_env_data, env_columns=env_columns2)
        .pipe(pre.add_target, target_days=week_2) # Week Shipmentをズラす前に作成することに注意
        .pipe(pre.add_target2, target_days=week_2) # Week Shipmentをズラす前に作成することに注意
        .pipe(pre.shift_unchange_name, shift_columns=['Week Shipment'], shift_days=-7) # Week Shipmentは1週間前のデータを使用
        .pipe(pre.categorize_columns, cat_columns=cat_columns)
)

df_week3 = (
        df_week3.pipe(pre.change_column_name)
        .pipe(pre.set_index_date)
        .pipe(pre.add_pred_rows, today=today, week_number_li=week_number_li, temp_li=temp_li, target_days=week_3)
        .pipe(pre.add_status, today=today, target_days=week_3, calender_path=calender_path)
        .pipe(pre.add_weekday)
        .pipe(pre.add_weekly_total, columns=weekly_total_columns)
        .pipe(pre.add_weekly_mean, columns=weekly_mean_columns) 
        .pipe(pre.add_week_status2)
        .pipe(pre.shift_unchange_name, shift_columns=shift_unchange_name_columns, shift_days=week_3)
        .pipe(pre.shift_change_name, shift_columns=shift_change_name_columns, shift_days=week_3)
        .pipe(pre.change_env_data, env_columns=env_columns3)
        .pipe(pre.add_target, target_days=week_3) # Week Shipmentをズラす前に作成することに注意
        .pipe(pre.add_target2, target_days=week_3) # Week Shipmentをズラす前に作成することに注意
        .pipe(pre.shift_unchange_name, shift_columns=['Week Shipment'], shift_days=-7) # Week Shipmentは1週間前のデータを使用
        .pipe(pre.categorize_columns, cat_columns=cat_columns)
)
# df_week2[['Temp-14', 'Week Temp-14']].tail(50)

           date  出荷量(ケース)  定植本数  平均気温  最高気温  最低気温 日照時間  AIRTMP  GLBRAD  \
2464 2024-04-30    2089.0     0  22.7   NaN   NaN  NaN     NaN     NaN   
2465 2024-05-01    2251.0     0  18.4   NaN   NaN  NaN     NaN     NaN   
2466 2024-05-02    2015.0     0  16.1   NaN   NaN  NaN     NaN     NaN   
2467 2024-05-03    1808.0     0  17.5   NaN   NaN  NaN     NaN     NaN   
2468 2024-05-04    1599.0     0  18.6   NaN   NaN  NaN     NaN     NaN   

      GLBRAD_30MIN  降水量  Week Number Week Start   Week End  
2464           NaN  NaN           40 2024-04-28 2024-05-04  
2465           NaN  NaN           40 2024-04-28 2024-05-04  
2466           NaN  NaN           40 2024-04-28 2024-05-04  
2467           NaN  NaN           40 2024-04-28 2024-05-04  
2468           NaN  NaN           40 2024-04-28 2024-05-04  


In [17]:
df_week1_ = df_week1[-output_len-week_1:-week_1]
df_week2_ = df_week2[-output_len-week_2:-week_2]
df_week3_ = df_week3[-output_len-week_3:-week_3]

In [18]:
data_week1_ = (
    df_week1_.pipe(pre.normalize_columns, df_=df_week1, feature_path=feature_path1)
    .pipe(pre.sort_columns, feature_path=feature_path1)
    .pipe(pre.fillna_mean)
)

data_week2_ = (
    df_week2_.pipe(pre.normalize_columns, df_=df_week2, feature_path=feature_path2)
    .pipe(pre.sort_columns, feature_path=feature_path2)
    .pipe(pre.fillna_mean)
)

data_week3_= (
    df_week3_.pipe(pre.normalize_columns, df_=df_week3, feature_path=feature_path3)
    .pipe(pre.sort_columns, feature_path=feature_path3)
    .pipe(pre.fillna_mean)
)

/home/code/Preprocessing.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numeric_columns] = (df[numeric_columns] - df_min) / range_
/home/code/Preprocessing.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numeric_columns] = (df[numeric_columns] - df_min) / range_
/home/code/Preprocessing.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [19]:
X_week1 = data_week1_.drop('Target2', axis=1).values
X_week2 = data_week2_.drop('Target2', axis=1).values
X_week3 = data_week3_.drop('Target2', axis=1).values

X_week1 = X_week1.reshape((X_week1.shape[0], X_week1.shape[1], 1))
X_week2 = X_week2.reshape((X_week2.shape[0], X_week2.shape[1], 1))
X_week3 = X_week3.reshape((X_week3.shape[0], X_week3.shape[1], 1))

In [20]:
week_1_model = load_model(model_path1)
week_2_model = load_model(model_path2)
week_3_model = load_model(model_path3)
# week_2_model.summary()

In [21]:
y_pred1 = week_1_model.predict(X_week1)
y_pred2 = week_2_model.predict(X_week2)
y_pred3 = week_3_model.predict(X_week3)

df_week1 = df_week1.reset_index()
df_week2 = df_week2.reset_index()
df_week3 = df_week3.reset_index()

df_week1.rename(columns={'index': 'Date'}, inplace=True)
df_week2.rename(columns={'index': 'Date'}, inplace=True)
df_week3.rename(columns={'index': 'Date'}, inplace=True)

result_1week = df_week1[['Date', 'Week Start', 'Week End']][-output_len-week_1:-week_1]
result_2week = df_week2[['Date', 'Week Start', 'Week End']][-output_len-week_2:-week_2]
result_3week = df_week3[['Date', 'Week Start', 'Week End']][-output_len-week_3:-week_3]

result_1week['pred_1week'] = y_pred1[:, 0] * df_week1['Target2'][950:1673].max() * df_week1['Week Status'][-output_len-week_1:-week_1].values
result_2week['pred_2week'] = y_pred2[:, 0] * df_week2['Target2'][950:1673].max() * df_week2['Week Status'][-output_len-week_2:-week_2].values
result_3week['pred_3week'] = y_pred3[:, 0] * df_week3['Target2'][950:1673].max() * df_week3['Week Status'][-output_len-week_3:-week_3].values

# df_week1['Week Status'][-output_len-week_1:-week_1].valuesと同じ長さで、全ての値が7のリストを作成
# pred_week_status = [7] * output_len
# result_1week['pred_1week'] = y_pred1[:, 0] * df_week1['Target2'][950:1673].max() * pred_week_status
# result_2week['pred_2week'] = y_pred2[:, 0] * df_week2['Target2'][950:1673].max() * pred_week_status
# result_3week['pred_3week'] = y_pred3[:, 0] * df_week3['Target2'][950:1673].max() * pred_week_status

In [22]:
output_df = result_1week.copy()
output_df['pred_2week'] = result_2week['pred_2week']
output_df['pred_3week'] = result_3week['pred_3week']
output_df

,Date,Week Start,Week End,pred_1week,pred_2week,pred_3week
2419,2024-03-16,2024-03-10,2024-03-15,7333.410645,9089.924316,13190.491699
2420,2024-03-17,2024-03-16,2024-03-23,8975.036011,12914.536011,14645.744873
2421,2024-03-18,2024-03-16,2024-03-23,9080.777588,12928.238647,14639.666016
2422,2024-03-19,2024-03-16,2024-03-23,8974.882202,12962.366211,14623.037598
2423,2024-03-20,2024-03-16,2024-03-23,8979.181519,12967.356445,14584.236816
2424,2024-03-21,2024-03-16,2024-03-23,9039.879761,12937.396240,14552.162598
2425,2024-03-22,2024-03-16,2024-03-23,9278.543701,12998.546265,14601.596680
2426,2024-03-23,2024-03-16,2024-03-23,9468.236694,12953.068481,14606.549316
2427,2024-03-24,2024-03-24,2024-03-30,14204.030518,14799.541504,13933.063354
2428,2024-03-25,2024-03-24,2024-03-30,13950.828247,14831.234619,13961.164185


In [23]:
# データフレームの作成
num = 6 # 実績値の数
# Week列において、下から見て行った時に7にぶつかるまでの行数を取得

date = output_df['Week Start'].iloc[-num*7::7].reset_index(drop=True)
this_year = df_week1['Week Shipment'].iloc[-42::7].reset_index(drop=True)
week_number = df['Week Number'].iloc[-num*7::7].reset_index(drop=True)

graph_df = pd.DataFrame({'Week Start': date, 'Week Number': week_number, '今年': this_year, '去年': [0]*num})

# 当週のデータを入力
graph_df['今年'].iloc[-1] = output_df['pred_1week'].iloc[-2]

# 予測値のデータフレーム作成
date = [graph_df['Week Start'].iloc[-1] + datetime.timedelta(days=7*i) for i in range(1, 4)]
this_year = [result_1week['pred_1week'].iloc[-1], result_2week['pred_2week'].iloc[-1], result_3week['pred_3week'].iloc[-1]]
week_number = [graph_df['Week Number'].iloc[-1] + i for i in range(1, 4)]

pred_df = pd.DataFrame({'Week Start': date, 'Week Number': week_number, '今年': this_year, '去年': [0]*3})

# データフレームの結合
graph_df = pd.concat([graph_df, pred_df], axis=0)
graph_df = graph_df.reset_index(drop=True)

# 去年のデータを取得
LastYear_Index = []
for i in range(len(graph_df)):
    df_bool = (graph_df["Week Start"].dt.year.iloc[i]-1 == df["date"].dt.year) & (df["Week Number"] == graph_df["Week Number"].iloc[i])
    LastYear_Index.append(df_bool[df_bool==True].index[0])

graph_df["去年"] = df_week1["Week Shipment"].shift(-7).iloc[LastYear_Index].values # 特徴量計算時に7日前のデータを使用しているため、7日シフト

graph_df.drop("Week Number", axis=1, inplace=True)

# graph_dfの「今年」と「去年」列を小数点第一位で四捨五入してint型に変換
graph_df['今年'] = graph_df['今年']+0.5
graph_df['去年'] = graph_df['去年']+0.5
graph_df['今年'] = graph_df['今年'].astype(int)
graph_df['去年'] = graph_df['去年'].astype(int)

graph_df.to_csv(output_path, index=False, encoding='shift-jis')

/tmp/ipykernel_14073/1887687531.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  graph_df['今年'].iloc[-1] = output_df['pred_1week'].iloc[-2]
